Hello world from (Scala) Spark!

In [1]:
println("Hello, world!")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1616422888890_0003,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Hello, world!


A simple word count

In [2]:
val sentence: String = "tre tigri contro tre tigri"
val res: Array[(String, Int)] = sc.parallelize(sentence.split(" ")).map((_, 1)).reduceByKey(_ + _).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

sentence: String = tre tigri contro tre tigri
res: Array[(String, Int)] = Array((tigri,2), (tre,2), (contro,1))


Save the result to HDFS

In [22]:
import org.apache.hadoop.fs.{FileSystem, Path}
val fs = FileSystem.get(sc.hadoopConfiguration) // get the file system
val outPutPath = new Path(path)
if (fs.exists(outPutPath)) { // delete the HDFS folder if exists
    fs.delete(outPutPath, true)
}

val hdfspath: String = "wordcount" // HDFS path
def writeandread(path: String) = {
    sc.parallelize(res).saveAsTextFile(path) // save the RDD
    val rdd = sc.textFile(path) // read it back
    rdd.collect() // print it 
}

writeandread(hdfspath)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.hadoop.fs.{FileSystem, Path}
fs: org.apache.hadoop.fs.FileSystem = DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_1600703682_22, ugi=livy (auth:SIMPLE)]]
outPutPath: org.apache.hadoop.fs.Path = wordcount
res28: AnyVal = true
hdfspath: String = wordcount
writeandread: (path: String)Array[String]
res32: Array[String] = Array((tigri,2), (tre,2), (contro,1))


... and to S3 as well

In [24]:
val s3bucket: String = "aws-emr-resources-604905954159-us-east-1"
val s3path: String = s"s3://${s3bucket}/wordcount"
writeandread(s3path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory s3://aws-emr-resources-604905954159-us-east-1/wordcount already exists
  at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
  at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:289)
  at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
  at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1090)
  at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
  at org.apache.spark.rdd.RDD.withScope(RDD.scala:388)
  at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1088)
  at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsH

[Exercise] Upload a file using Amazon SDK 

In [71]:
val key: String = "divinacommedia/inferno.txt"
try {
    val s3Client = AmazonS3ClientBuilder.standard().build()
    // s3Client.deleteObject(bucketName, key)
    s3Client.deleteObject(bucketName, ...) // Fill in the gaps, program it to upload the file content
} catch {
    case e: Throwable => e.printStackTrace()
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

key: String = divinacommedia/inferno.txt


From Spark to Hive

In [78]:
import org.apache.spark.sql.hive.HiveContext
val hiveContext = new HiveContext(sc)
import hiveContext.implicits._

sc.textFile(s"s3://${s3bucket}/inferno.txt").flatMap(_.split(" ")).filter(_.nonEmpty).map((_, 1)).reduceByKey(_ + _).toDF("word", "count").write.mode("overwrite").saveAsTable("inferno")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.hive.HiveContext
hiveContext: org.apache.spark.sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@7eb10552
import hiveContext.implicits._


In [79]:
sql("select word, count from inferno order by 2 desc").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+
|word|count|
+----+-----+
|   e| 3613|
| che| 3535|
|  la| 2261|
|  di| 1823|
|   a| 1805|
| non| 1338|
| per| 1319|
|  in| 1071|
|  si| 1042|
|  ?l|  958|
|  le|  777|
|  li|  753|
|  mi|  736|
|  s?|  706|
|  il|  652|
| pi?|  637|
| con|  623|
|   ?|  600|
|  de|  593|
|  da|  593|
+----+-----+
only showing top 20 rows

